In [117]:
import pandas as pd

In [118]:
missing_values = ["n/a", "na", "--", "", "XNA", "Unknown", "unknown"]

db = pd.read_csv("atms_data.csv", na_values=missing_values)
db.rename({'Amount                                                                   ':'Amount'}, axis=1, inplace=True)
db.drop(['Unnamed: 0'], axis=1, inplace=True)
db.drop(['capacity'], axis=1, inplace=True)
db['Date']=pd.to_datetime(db['Date'], format='%Y%m%d')
print("Shape of application data: ", db.shape)
print(db.head())

Shape of application data:  (573119, 4)
        Date   ATM ID  Number of Trxs  Amount
0 2018-02-01  3479212             199  235250
1 2018-02-01  1599756             105  237300
2 2018-02-01  2019227             487  539910
3 2018-02-01  3200101             354  466900
4 2018-02-01  5129042             224  432610


In [119]:
db = db[db['ATM ID']==2079266].set_index('Date').drop(['ATM ID'], axis=1)
complete_df = pd.DataFrame({'Date': pd.date_range(start=db.index.values.min(), end=db.index.values.max(), freq='D')})
merged_df = pd.merge(complete_df, db, on='Date', how='left').merge(holidays, on='Date', how='left')
merged_df.set_index('Date', inplace=True)
merged_df.index.freq = 'D'
complete_df = merged_df.interpolate(method='linear')
complete_df[7:16]

/var/folders/kw/dchp08qs3lqgjzpc8kwkl5xr0000gn/T/ipykernel_2346/568588054.py:6: FutureWarning: DataFrame.interpolate with object dtype is deprecated and will raise in a future version. Call obj.infer_objects(copy=False) before interpolating instead.
  complete_df = merged_df.interpolate(method='linear')


,Number of Trxs,Amount,Holiday Name,Holiday Type
Date,,,,
2018-02-08,77.0,128550.0,NaN,NaN
2018-02-09,195.0,273400.0,NaN,NaN
2018-02-10,209.0,328890.0,NaN,NaN
2018-02-11,218.0,317950.0,NaN,NaN
2018-02-12,174.0,259750.0,NaN,NaN
2018-02-13,82.0,114450.0,NaN,NaN
2018-02-14,220.0,312250.0,International Valentine's,Not A Public Holiday
2018-02-15,238.0,391450.0,NaN,NaN
2018-02-16,105.0,155250.0,NaN,NaN


In [120]:
holidays = pd.read_csv("2018 Holidays.csv", na_values=missing_values)
holidays['Date']=pd.to_datetime(holidays['Date'], format='%d/%m/%Y')
print(holidays.head())

        Date               Holiday Name          Holiday Type
0 2018-01-07           Coptic Christmas     Religious Holiday
1 2018-01-25  Revolution Day January 25      National Holiday
2 2018-02-14  International Valentine's  Not A Public Holiday
3 2018-03-21               Mother's Day  Not A Public Holiday
4 2018-04-08       Coptic Easter Sunday     Religious Holiday


In [121]:
complete_df['day'] = complete_df.index.day
complete_df['month'] = complete_df.index.month
complete_df['year'] = complete_df.index.year
complete_df['weekday'] = complete_df.index.weekday
complete_df['dayName']=complete_df.index.day_name()
complete_df['is_weekend'] = complete_df.index.weekday.isin([4,5]).astype(int)
complete_df['is_weekday'] = complete_df.index.weekday.isin([0,1,2,3,6]).astype(int)
complete_df['days_in_month'] = complete_df.index.days_in_month
complete_df['is_holiday'] = complete_df['Holiday Name'].notnull().astype(int)
complete_df['is_dayoff']=(complete_df['is_weekend'] | complete_df['is_holiday']).astype(int)

In [122]:
complete_df.head()

,Number of Trxs,Amount,Holiday Name,Holiday Type,day,month,year,weekday,dayName,is_weekend,is_weekday,days_in_month,is_holiday,is_dayoff
Date,,,,,,,,,,,,,,
2018-02-01,606.0,852790.0,NaN,NaN,1,2,2018,3,Thursday,0,1,28,0,0
2018-02-02,111.0,164000.0,NaN,NaN,2,2,2018,4,Friday,1,0,28,0,1
2018-02-03,157.0,293970.0,NaN,NaN,3,2,2018,5,Saturday,1,0,28,0,1
2018-02-04,324.0,493650.0,NaN,NaN,4,2,2018,6,Sunday,0,1,28,0,0
2018-02-05,398.0,624720.0,NaN,NaN,5,2,2018,0,Monday,0,1,28,0,0


In [123]:
complete_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 334 entries, 2018-02-01 to 2018-12-31
Freq: D
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Number of Trxs  334 non-null    float64
 1   Amount          334 non-null    float64
 2   Holiday Name    21 non-null     object 
 3   Holiday Type    21 non-null     object 
 4   day             334 non-null    int32  
 5   month           334 non-null    int32  
 6   year            334 non-null    int32  
 7   weekday         334 non-null    int32  
 8   dayName         334 non-null    object 
 9   is_weekend      334 non-null    int64  
 10  is_weekday      334 non-null    int64  
 11  days_in_month   334 non-null    int32  
 12  is_holiday      334 non-null    int64  
 13  is_dayoff       334 non-null    int64  
dtypes: float64(2), int32(5), int64(4), object(3)
memory usage: 32.6+ KB


In [126]:
complete_df.to_csv('selected_atm.csv', index=True, header=True) 

In [127]:
pd.read_csv('selected_atm.csv', index_col='Date', parse_dates=True).head()


,Number of Trxs,Amount,Holiday Name,Holiday Type,day,month,year,weekday,dayName,is_weekend,is_weekday,days_in_month,is_holiday,is_dayoff
Date,,,,,,,,,,,,,,
2018-02-01,606.0,852790.0,NaN,NaN,1,2,2018,3,Thursday,0,1,28,0,0
2018-02-02,111.0,164000.0,NaN,NaN,2,2,2018,4,Friday,1,0,28,0,1
2018-02-03,157.0,293970.0,NaN,NaN,3,2,2018,5,Saturday,1,0,28,0,1
2018-02-04,324.0,493650.0,NaN,NaN,4,2,2018,6,Sunday,0,1,28,0,0
2018-02-05,398.0,624720.0,NaN,NaN,5,2,2018,0,Monday,0,1,28,0,0
